In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-5f8dnqg3
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-5f8dnqg3
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=ef489fb61d7487428fc7088f4aa3ffb469ffaeb28bc0fc9ecaacce1c06b9fe3a
  Stored in directory: /tmp/pip-ephem-wheel-cache-luvk3k22/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import clip
from torchvision import models

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/img_align_celeba.zip'  # Adjust path as needed
extract_path = '/content/data/'  # Path to extract files to

# Create the extract directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f'Files extracted to {extract_path}')


Files extracted to /content/data/


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 64
EPOCHS = 1
LEARNING_RATE = 1e-4

# Load CLIP model
clip_model, preprocess = clip.load("ViT-B/32", device=DEVICE)
clip_model.eval()

# Load ProGAN from PyTorch Hub
progan = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 'PGAN', model_name='celebAHQ-256', pretrained=True, useGPU=torch.cuda.is_available())


100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 80.9MiB/s]
Downloading: "https://github.com/facebookresearch/pytorch_GAN_zoo/zipball/hub" to /root/.cache/torch/hub/hub.zip
Downloading: "https://dl.fbaipublicfiles.com/gan_zoo/PGAN/celebaHQ_s6_i80000-6196db68.pth" to /root/.cache/torch/hub/checkpoints/celebaHQ_s6_i80000-6196db68.pth
100%|██████████| 264M/264M [00:01<00:00, 211MB/s]


Average network found !


In [ ]:
import torch.nn.functional as F

In [ ]:
class MappingNetwork(nn.Module):
    def __init__(self, input_dim=512, latent_dim=512):
        super(MappingNetwork, self).__init__()
        layers = []
        for _ in range(12):
            layers.append(nn.Linear(input_dim, input_dim))
            layers.append(nn.LeakyReLU(0.2))
        layers.append(nn.Linear(input_dim, latent_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [ ]:
import gdown

In [ ]:
mapping_network = MappingNetwork().to(DEVICE)


In [ ]:

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class RawImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        """
        Args:
            image_dir (str): Directory with raw images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(('jpg', 'jpeg', 'png'))]  # Filter image files
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_name = self.image_files[idx]
        image_path = os.path.join(self.image_dir, image_name)

        # Open the image
        image = Image.open(image_path).convert("RGB")  # Ensure 3-channel RGB

        # Apply transformations if defined
        if self.transform:
            image = self.transform(image)

        return image

# Define transformations (optional)
# Define any transformations you want to apply to your images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])


# Initialize the dataset
dataset = RawImageDataset(image_dir='/content/data/img_align_celeba', transform=transform)

# Create a DataLoader
batch_size = 16
celeba_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Verify the first batch of data
# data_iter = iter(dataloader)
# images = data_iter.next()
# print(images.shape)  # Should print (batch_size, 3, 128, 128) if using the transformations above


In [ ]:
len(celeba_loader)

12663

In [ ]:
# prompt: out of the 12663 batches, i want to train only on the first 5000 batches

# ... (Your existing code)

# Initialize the dataset
dataset = RawImageDataset(image_dir='/content/data/img_align_celeba', transform=transform)

# Create a DataLoader with limited number of batches
num_batches_to_use = 5000
subset_indices = list(range(min(num_batches_to_use * batch_size, len(dataset))))  # Calculate indices to include

subset_dataset = torch.utils.data.Subset(dataset, subset_indices)

celeba_loader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=True)

# ... (Rest of your code)

In [ ]:
len(celeba_loader)

5000

In [ ]:
IMAGE_SIZE = 224
discriminator = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(0.2),
    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
    nn.LeakyReLU(0.2),
    nn.Flatten(),
    nn.Linear(128 * (IMAGE_SIZE // 4) ** 2, 1)  # Single scalar output
).to(DEVICE)

In [ ]:
# prompt: write code to check if there is a .pth file available for discriminator as well as mapping network if yes load the weights to it and use it in further part of the code

import os

# Check for discriminator .pth file
discriminator_path = 'discriminator.pth'  # Replace with your actual path
if os.path.exists(discriminator_path):
    print("Discriminator weights found. Loading...")
    discriminator.load_state_dict(torch.load(discriminator_path, map_location=DEVICE))
    discriminator.eval()  # Set to evaluation mode
else:
    print("Discriminator weights not found.")

# Check for mapping network .pth file
mapping_network_path = 'mapping_network_progan.pth'  # Replace with your actual path
if os.path.exists(mapping_network_path):
    print("Mapping Network weights found. Loading...")
    mapping_network.load_state_dict(torch.load(mapping_network_path, map_location=DEVICE))
    mapping_network.eval()  # Set to evaluation mode
else:
    print("Mapping Network weights not found.")

Discriminator weights not found.
Mapping Network weights not found.


In [ ]:
def gradient_penalty(discriminator, real_data, fake_data):
    batch_size = real_data.size(0)
    alpha = torch.rand(batch_size, 1, 1, 1).to(DEVICE)
    interpolates = alpha * real_data + (1 - alpha) * fake_data
    interpolates.requires_grad_(True)

    disc_interpolates = discriminator(interpolates)
    grad_outputs = torch.ones_like(disc_interpolates).to(DEVICE)

    gradients = torch.autograd.grad(
        outputs=disc_interpolates,
        inputs=interpolates,
        grad_outputs=grad_outputs,
        create_graph=True,
        retain_graph=True
    )[0]

    gradients = gradients.view(batch_size, -1)
    gradient_norm = gradients.norm(2, dim=1)
    penalty = ((gradient_norm - 1) ** 2).mean()
    return penalty

In [ ]:

criterion_reconstruction = nn.MSELoss()
criterion_identity = nn.CosineSimilarity(dim=1)

# Define a simple discriminator for adversarial loss


# Perceptual Loss Model (LPIPS)
perceptual_model = models.alexnet(pretrained=True).features.to(DEVICE).eval()

# Diversity Loss
def diversity_loss(feature_1, feature_2, img_1, img_2, scaling_factor=10.0):
    img_1_flat = img_1.view(img_1.size(0), -1)
    img_2_flat = img_2.view(img_2.size(0), -1)
    dist_features = torch.norm(feature_1 - feature_2, p=1, dim=1)
    dist_images = torch.norm(img_1_flat - img_2_flat, p=1, dim=1)
    diversity_loss = torch.mean(dist_features / (dist_images + 1e-8))
    return scaling_factor * diversity_loss  # Scaled for impact


# Optimizers
optimizer = optim.Adam(mapping_network.parameters(), lr=LEARNING_RATE)
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:05<00:00, 43.1MB/s]


In [ ]:
for epoch in range(EPOCHS):
    for i, images in enumerate(celeba_loader):
        images = images.to(DEVICE)

        # Train Discriminator
        for _ in range(5):
            clip_features = clip_model.encode_image(images).float().to(DEVICE)
            latent_vectors = mapping_network(clip_features).to(DEVICE)

            # Noise injection during generation
            noise = torch.randn_like(latent_vectors).to(DEVICE)
            latent_vectors = latent_vectors + 0.05 * noise

            generated_images = progan.test(latent_vectors)
            generated_images = generated_images.to(DEVICE)
            generated_images_resized = F.interpolate(
                generated_images, size=(IMAGE_SIZE, IMAGE_SIZE), mode='bilinear', align_corners=False
            )

            real_scores = discriminator(images)
            fake_scores = discriminator(generated_images_resized.detach())
            gp = gradient_penalty(discriminator, images, generated_images_resized)
            loss_discriminator = -(real_scores.mean() - fake_scores.mean()) + 10 * gp

            discriminator_optimizer.zero_grad()
            loss_discriminator.backward()
            discriminator_optimizer.step()

        # Train Generator
        optimizer.zero_grad()
        latent_vectors = mapping_network(clip_features).to(DEVICE)
        generated_images = progan.test(latent_vectors).to(DEVICE)
        generated_images_resized = F.interpolate(
            generated_images, size=(IMAGE_SIZE, IMAGE_SIZE), mode='bilinear', align_corners=False
        )
        loss_generator = -discriminator(generated_images_resized).mean()

        noise = torch.randn_like(clip_features).to(DEVICE)
        perturbed_features = clip_features + noise
        perturbed_latents = mapping_network(perturbed_features)
        diverse_images = progan.test(perturbed_latents).to(DEVICE)
        loss_diversity = diversity_loss(
            clip_features, perturbed_features, generated_images, diverse_images
        )

        loss_reconstruction = criterion_reconstruction(
            F.interpolate(generated_images, size=(IMAGE_SIZE, IMAGE_SIZE), mode='bilinear', align_corners=False),
            images
        )
        loss_identity = 1 - criterion_identity(
            clip_features,
            clip_model.encode_image(generated_images_resized).float()
        )
        with torch.no_grad():
            features_real = perceptual_model(images)
            features_fake = perceptual_model(F.interpolate(generated_images, size=(IMAGE_SIZE, IMAGE_SIZE), mode='bilinear', align_corners=False))
        loss_perceptual = torch.mean((features_real - features_fake) ** 2)

        total_loss = (
            loss_reconstruction +
            0.1 * loss_generator +
            0.1 * loss_identity +
            0.1 * loss_diversity +
            0.1*loss_perceptual
        ).mean()
        total_loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{EPOCHS}, D Loss: {loss_discriminator.item():.4f}, G Loss: {loss_generator.item():.4f}, Total Loss: {total_loss.item():.4f}")

# Save models
torch.save(mapping_network.state_dict(), "mapping_network_wgan_epoch1.pth")
torch.save(discriminator.state_dict(), "discriminator_wgan_epoch1.pth")

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/1, D Loss: -254.0592, G Loss: 6.8869, Total Loss: 6.2487


In [ ]:
# prompt: also save the weights of discriminator as we are saving of the mapping network

# Save the trained discriminator
torch.save(discriminator.state_dict(), "discriminator_newweights.pth")

In [ ]:
import torch
from PIL import Image
import clip

# Load CLIP model
clip_model, preprocess = clip.load("ViT-B/32", device=DEVICE)
clip_model.eval()

# Load ProGAN from PyTorch Hub
progan = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 'PGAN', model_name='celebAHQ-256', pretrained=True, useGPU=torch.cuda.is_available())

# Load the trained Mapping Network
class MappingNetwork(nn.Module):
    def __init__(self, input_dim=512, latent_dim=512):
        super(MappingNetwork, self).__init__()
        layers = []
        for _ in range(12):
            layers.append(nn.Linear(input_dim, input_dim))
            layers.append(nn.LeakyReLU(0.2))
        layers.append(nn.Linear(input_dim, latent_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

mapping_network = MappingNetwork().to(DEVICE)
# mapping_network.load_state_dict(torch.load("/content/mapping_network_wgan_epoch1.pth", map_location=DEVICE))
mapping_network.eval()

# Function for Generating Images from Text
def generate_image_from_text(text, output_path="generate_image.png"):
    # Encode text using CLIP
    with torch.no_grad():
        text_features = clip_model.encode_text(clip.tokenize([text]).to(DEVICE)).float()

    # Map text features to ProGAN latent space
    with torch.no_grad():
        latent_vector = mapping_network(text_features)

    # Generate image using ProGAN
    with torch.no_grad():
        generated_image = progan.test(latent_vector)

    # Convert image to PIL format and save
    generated_image = (generated_image[0].clamp(-1, 1) + 1) / 2  # Normalize to [0, 1]
    generated_image = (generated_image * 255).permute(1, 2, 0).cpu().numpy().astype('uint8')
    Image.fromarray(generated_image).save(output_path)
    print(f"Image saved at {output_path}")

# Example Usage
text_description = "black man"
generate_image_from_text(text_description)


Using cache found in /root/.cache/torch/hub/facebookresearch_pytorch_GAN_zoo_hub


Average network found !
Image saved at generate_image.png
